In [28]:
import json
from pathlib import Path
import random
import yaml
import os

from mtg.bot import mtg_chain
from mtg.history import ChatHistory
from mtg.objects import MessageType


with open("configs/config.yaml", "r") as infile:
    config = yaml.load(infile, Loader=yaml.FullLoader)

os.environ["OPENAI_API_KEY"] = config["open_ai_token"]

history = ChatHistory()
llm = mtg_chain.create_llm(model_name="gpt-4-0125-preview", temperature=1.0)

In [29]:
DATA_PATH = Path("../data/etl/")


In [30]:
def create_dataset(data:dict):
    return [
        {
            "question": q["metadata"]["question"],
            "ground_truth": q["metadata"]["answer"],
        }
        for q in data
    ]


def ask(question) -> str: 
    # create chain
    memory = mtg_chain.create_chat_memory(llm=llm, max_token_limit=1000)
    rules_chain = mtg_chain.create_rules_chain(llm=llm, memory=memory)

    # add cards data 
    cards = history.data_service.get_cards(question, k=10, lasso_threshold=0.2)
    _, matched_cards = history.replace_card_names_with_urls(
        text=question, cards=cards, message_type=MessageType.RULES
    )

    # add rules data 
    rules = history.data_service.get_rules(question)
    keywords = []
    for card in matched_cards:
        keywords.extend(card.keywords)
    if keywords:
        rules.extend(history.data_service.get_rules(".".join(keywords)))
    
    # combine rag chunks
    contexts = []
    for card in cards:
        contexts.append(card.to_text(include_price=False))
    for rule in rules:
        contexts.append(rule.text)

    answer = rules_chain.invoke(
        {
            "human_input": question,
            "card_data": "\n".join([card.to_text(False) for card in cards]),
            "rules_data": "\n".join([rule.text for rule in rules]),
        }
    )

    return answer.content, contexts


# Rulesguru

In [ ]:

with open(DATA_PATH / "raw/documents/rulesguru.json", "r", encoding="utf-8") as infile:
    rules_guru_data = json.load(infile)

In [4]:
levels = ["0", "1", "2"]
number_of_questions = 5 

full_dataset = {}
for level in levels:
    full_dataset[f"level_{level}"] = []
    filtered_data = [d for d in rules_guru_data if d["metadata"]["level"] == level]
    filtered_data = random.choices(filtered_data, k=number_of_questions)
    dataset = create_dataset(filtered_data)
    for document in dataset:
        answer, context = ask(document["question"])
        document["answer"] = answer 
        document["contexts"] = context
        full_dataset[f"level_{level}"].append(document)

2024-03-12 10:15:44,065 - mtg.history.spacy_utils - INFO - matched 2 cards: ['Hyena Pack', 'Satyr Piper'] (spacy_utils.py:97)
2024-03-12 10:15:46,812 - mtg.history.spacy_utils - INFO - matched 2 cards: ["Johann's Stopgap", 'Narset, Parter of Veils'] (spacy_utils.py:97)
2024-03-12 10:15:50,669 - mtg.history.spacy_utils - INFO - matched 3 cards: ['Karn, Legacy Reforged', 'Approach of the Second Sun', 'Karn Liberated'] (spacy_utils.py:97)
2024-03-12 10:15:59,229 - mtg.history.spacy_utils - INFO - matched 3 cards: ['Archive Trap', 'Mindlock Orb', 'Prismatic Vista'] (spacy_utils.py:97)
2024-03-12 10:16:04,641 - mtg.history.spacy_utils - INFO - matched 1 cards: ['Gemini Engine'] (spacy_utils.py:97)
2024-03-12 10:16:07,567 - mtg.history.spacy_utils - INFO - matched 3 cards: ['Athreos, Shroud-Veiled', 'Dance of the Dead', 'Athreos, God of Passage'] (spacy_utils.py:97)
2024-03-12 10:16:22,046 - mtg.history.spacy_utils - INFO - matched 3 cards: ['Mishra, Artificer Prodigy', "Mishra's Bauble", 'S

[{'question': "Avery controls a Moritte of the Frost that's a copy of Garruk Relentless. It has 3 loyalty counters. They activate its first ability targeting Coral Merfolk, causing it to lose 2 loyalty as it resolves. What happens next?",
  'ground_truth': "The game ends in a draw unless a player chooses to interact.\n\nMoritte of the Frost triggers to transform itself. When the trigger resolves, it won't be able to transform since it is not a transforming double-faced card.. Since the condition is still true, the triggered ability will trigger again and so on.. This is a loop that does not change the game state and doesn't need players to make choices since all actions in it are forced. If no player wishes to interact with this (for example by destroying Moritte of the Frost while its trigger is on the stack), the game is a draw..",
  'answer': "When Avery controls a Moritte of the Frost that's a copy of Garruk Relentless and it has 3 loyalty counters, activating its first ability tha

In [ ]:
with open(
    DATA_PATH / "processed/evaluation/rulesguru.json", "w", encoding="utf-8"
) as outfile:
    json.dump(full_dataset, outfile)

# Stackexchange

In [11]:
with open(DATA_PATH / "processed/documents/stackexchange.json", "r", encoding="utf-8") as infile:
    stackexchange_data = json.load(infile)

In [19]:
number_of_questions, full_dataset = 5, {}

full_dataset[f"stackexchange"] = []
filtered_data = random.choices(stackexchange_data, k=number_of_questions)
dataset = create_dataset(filtered_data)
for document in dataset:
    answer, context = ask(document["question"])
    document["answer"] = answer 
    document["contexts"] = context
    full_dataset[f"stackexchange"].append(document)

2024-03-12 10:46:08,434 - mtg.history.spacy_utils - INFO - matched 0 cards: [] (spacy_utils.py:97)
2024-03-12 10:46:21,861 - mtg.history.spacy_utils - INFO - matched 2 cards: ['Shipbreaker Kraken', 'Mutavault'] (spacy_utils.py:97)
2024-03-12 10:46:28,410 - mtg.history.spacy_utils - INFO - matched 0 cards: [] (spacy_utils.py:97)
2024-03-12 10:46:34,885 - mtg.history.spacy_utils - INFO - matched 0 cards: [] (spacy_utils.py:97)
2024-03-12 10:46:42,013 - mtg.history.spacy_utils - INFO - matched 2 cards: ['Sliver Overlord', "Conjurer's Closet"] (spacy_utils.py:97)


In [20]:
with open(
    DATA_PATH / "processed/evaluation/stackexchange.json", "w", encoding="utf-8"
) as outfile:
    json.dump(full_dataset, outfile)


# Expert Knowledge

In [31]:
with open(DATA_PATH / "raw/documents/expert_knowledge.json", "r", encoding="utf-8") as infile:
    expert_data = json.load(infile)
expert_data

[{'question': 'If I control Solphim, Mayhem Dominus and Imodane, the Pyrohammer, and I target an opponent’s creature with a burn spell what happens?',
  'answer': 'The damage of the burn spell gets doubled because of Solphim, Mayhem Dominus. Then Imodane will deal that much damage to every opponent. Solphim, Mayhem Dominus triggered ability acts as a replacement effect for the damage of the burn spell.'},
 {'question': "what happens to Urza's Saga when Blood Moon is played?",
  'answer': "Saga's as a rule are sacrificed as a state based action when the number of counters are equal or greater than the number of chapter abilities (and as long as there isn't a related chapter trigger on the stack). So when Urzas saga loses all of it's abilities due to blood moon or a similar effect it's a saga with zero chapter abilities. Which means it's sacrificed as a state based action."},
 {'question': 'What happens if i controll two creatures and Repercussion and my opponent controlls 2 creatures. W

In [32]:
full_dataset = {}
full_dataset[f"expert_knowledge"] = []

for document in expert_data:
    document["ground_truth"] = document["answer"]
    answer, context = ask(document["question"])
    document["answer"] = answer 
    document["contexts"] = context
    full_dataset[f"expert_knowledge"].append(document)

2024-03-12 11:03:36,995 - mtg.history.spacy_utils - INFO - matched 1 cards: ['Imodane, the Pyrohammer'] (spacy_utils.py:97)
2024-03-12 11:03:53,273 - mtg.history.spacy_utils - INFO - matched 2 cards: ["Urza's Rage", 'Blood Moon'] (spacy_utils.py:97)
2024-03-12 11:04:01,982 - mtg.history.spacy_utils - INFO - matched 2 cards: ['Repercussion', 'Blasphemous Act'] (spacy_utils.py:97)
2024-03-12 11:04:42,842 - mtg.history.spacy_utils - INFO - matched 2 cards: ['Blazing Shoal', 'Overblaze'] (spacy_utils.py:97)
2024-03-12 11:04:54,495 - mtg.history.spacy_utils - INFO - matched 2 cards: ['Sudden Spoiling', 'Saruman of Many Colors'] (spacy_utils.py:97)
2024-03-12 11:05:07,742 - mtg.history.spacy_utils - INFO - matched 1 cards: ['Pantlaza, Sun-Favored'] (spacy_utils.py:97)
2024-03-12 11:05:14,805 - mtg.history.spacy_utils - INFO - matched 4 cards: ['Teysa, Opulent Oligarch', 'Ratadrabik of Urborg', 'Teysa, Envoy of Ghosts', 'Teysa Karlov'] (spacy_utils.py:97)


In [24]:
with open(
    DATA_PATH / "processed/evaluation/expert_knowledge.json", "w", encoding="utf-8"
) as outfile:
    json.dump(full_dataset, outfile)


In [ ]:
other_questions = [
    "Can you explain how the stack works in Magic: The Gathering?",
    "What happens if I cast a spell with lifelink and my opponent counters it?",
    "If I control two 'Platinum Angel' cards and lose life below zero, do I lose the game?",
    "How does 'double strike' work when combined with 'trample'?",
    "How does 'deathtouch' work when combined with 'trample'?",
    "What are the rules for assigning damage to multiple blockers?",
    "Can I activate an ability of a creature the turn it comes into play?",
    "How do 'hexproof' and 'shroud' differ in Magic: The Gathering?",
    "What happens if a card's ability triggers during my opponent's end step?",
    "If I have 'Leyline of Sanctity' in play, can my opponent target me with discard spells?",
    "Can you explain how 'indestructible' and 'destroy' effects interact?",
    "Can you explain the basics of how to play Magic?",
]

